## Series y DataFrames

- [Guía de markdown - enlace](https://www.markdownguide.org/cheat-sheet/)
- [Video de YouTube](https://youtu.be/eobhLRplzk8)

En este notebook veremos las dos principales estructuras de datos de pandas, `Series` y `DataFrames`.

Una `Series` es un arreglo (array) unidimensional y un `DataFrame` es un array de dos dimensiones usado para representar data tabular, donde cada columna seria una `Series`.

Muchos de los métodos usados para los `DataFrames` sirven para las `Series` por lo que nos enfocaremos en este último.

### Ejemplo de `Series`

In [1]:
import pandas as pd

Podemos usar el ejemplo del notebook anterior para contruis una `Series`.

In [2]:
# datos

series = {
    "colores": ["rojo", "naranja", "verde", "amarillo"],
    "frutas": ["manzana", "mandarina", "kiwi", "pina"]
}


In [3]:
pd.Series(series)

colores    [rojo, naranja, verde, amarillo]
frutas     [manzana, mandarina, kiwi, pina]
dtype: object

La razón por la que no se ve como se espera es porque las `Series` son unidimensionales y la data que introdujimos tiene dos dimensiones por lo que ha tomados las keys como `index` y el resto como los valores.

Las `Series` a pesar de ser unidimensional, tienen varios atributos, dos de ellos son el nombre de la `Series` y el índice o `index`.

Ahora definámoslo correctamente.

In [4]:
pd.Series(
    ["rojo", "naranja", "verde", "amarillo"],
    name="colores",
    index=["manzana", "mandarina", "kiwi", "pina"],
)

manzana          rojo
mandarina     naranja
kiwi            verde
pina         amarillo
Name: colores, dtype: object

No es necesario definir un `index`, la mayoría de las veces no es definido in este es creado automáticamente de manera incremental desde cero.

In [5]:
colores = pd.Series(
    ["rojo", "naranja", "verde", "amarillo"],
    name="colores",
)

colores

0        rojo
1     naranja
2       verde
3    amarillo
Name: colores, dtype: object

In [6]:
frutas = pd.Series(
    ["manzana", "mandarina", "kiwi", "pina"],
    name="frutas",
)

frutas

0      manzana
1    mandarina
2         kiwi
3         pina
Name: frutas, dtype: object

Ahora podemos definir un `DataFrame` con ambas `Series` con la siguiente sintaxis:

In [7]:
# conviertes frutas en un dataframe
frutas_y_colores = frutas.to_frame()
# añade la columna colores
frutas_y_colores["colores"] = colores

frutas_y_colores

,frutas,colores
0,manzana,rojo
1,mandarina,naranja
2,kiwi,verde
3,pina,amarillo


Pandas unirá a las dos `Series` usando el `index` como referencia. No te preocupes si este último paso no quedó claro, fue solo una intro, más adelante trataremos con los `DataFrames`.


Puedes seguir la [documentación oficial](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) de las `Series`.

En data tabular, generalmente cada columna representa un solo tipo de datos, edad seria tipo `int`, dinero tipo `float`, etc. Pandas permite tener columnas mixtas, esto podría ser bueno en un caso especial pero en general es mala idea. Pandas permite hacer operaciones vectorizadas que son mucho más rápidas que si las hiciese elemento por elemento, esto se debe a que pandas por defecto usa NumPy y una `Series`se comporta como un `array` de NumPy.

Veamos la diferencia de vectorizar vs. un for loop.

In [8]:
series = pd.Series(range(1000000), name="naturales")
series.head()

0    0
1    1
2    2
3    3
4    4
Name: naturales, dtype: int64

In [9]:
%%timeit
# multiplicar por 2
series_two_times = series.mul(2)


255 μs ± 9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
# tambien puedes multiplicarlo directamente con el mismo resultado
series * 2

0               0
1               2
2               4
3               6
4               8
           ...   
999995    1999990
999996    1999992
999997    1999994
999998    1999996
999999    1999998
Name: naturales, Length: 1000000, dtype: int64

In [11]:
%%timeit
# ahora con un for loop
for i in series:
    i * 2

55.2 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Vemos que usando vectorización el tiempo fue menor que usando el for loop. En pandas por lo general siempre es mejor aplicar un método que for loops.

Podemos definir los tipos de datos en Pandas:

In [12]:
# Enteros
pd.Series(range(5), dtype="int64")

0    0
1    1
2    2
3    3
4    4
dtype: int64

In [13]:
# floats
pd.Series(range(5), dtype="float64")

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [14]:
# boolean
pd.Series([True, False, True, False, True], dtype="bool")

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [15]:
# strings
pd.Series(["manzana", "mandarina", "kiwi", "pina"], dtype="string")

0      manzana
1    mandarina
2         kiwi
3         pina
dtype: string

In [16]:
# datetypes
pd.Series(["2025-02-01", "2025-02-02", "2025-02-03", "2025-02-04", "2025-02-05"], dtype="datetime64[ns]")

0   2025-02-01
1   2025-02-02
2   2025-02-03
3   2025-02-04
4   2025-02-05
dtype: datetime64[ns]

In [17]:
# categorical
pd.Series(["camisa", "pantalon", "medias", "zapatos"], dtype="category")

0      camisa
1    pantalon
2      medias
3     zapatos
dtype: category
Categories (4, object): ['camisa', 'medias', 'pantalon', 'zapatos']

El tipo de dato `Category` permite pues, tener categorías, usa menos memoria que `strings` porque hace un mapeo e los valores únicos con un número. Por ejemplo, camisa es 0, pantalon es 1, etc. Si hay 1000 filas con camisa, por detrás está guardando 0 en la memoria y la lista de categoría solo guarda el nombre una vez.

Cuando tengamos un valor nulo, pandas usará de NumPy el `np.nan`.

In [18]:
import numpy as np

pd.Series([1, 2, 3, np.nan, 4, 5])

0    1.0
1    2.0
2    3.0
3    NaN
4    4.0
5    5.0
dtype: float64

Un detalle de esto que `np.nan` es un objeto de tipo `float` por lo que pandas aplicará trucos internos para solventar esto en columnas con otro tipo de datos en detrimento del performance. Una columna de `int` será de tipo `float` y una columna de `string` será de tipo `object` que es un tipo de datos general que pandas usa como salvavidas que permite tipos mixtos.

In [19]:
pd.Series(["manzana", "mandarina", np.nan, "pina"])

0      manzana
1    mandarina
2          NaN
3         pina
dtype: object

Pandas 2 tiene otro backend distinto a NumPy, PyArrow. Más adelante en lo posible usaremos este backend dado que será el usado por defecto en pandas 3 debido a ventajas como poder tener valores nulos con cualquier tipo de datos sin aplicar "trucos" sin disminuir performance, por ahora no nos preocupamos por ello.

In [20]:
!pip install pyarrow # es necesario tener pyarrow instalado

# si no funciona después de instalar pyarrow, has restart en el kernel de jupyter


In [21]:
# para tipo int
pd.Series([1, 2, 3, np.nan, 4, 5], dtype="int64[pyarrow]")

0       1
1       2
2       3
3    <NA>
4       4
5       5
dtype: int64[pyarrow]

In [22]:
# para strings
pd.Series(["manzana", "mandarina", np.nan, "pina"], dtype="string[pyarrow]")

0      manzana
1    mandarina
2         <NA>
3         pina
dtype: string

## Operaciones básicas

### Filtrar

Para filtrar en pandas se puede usar una máscara que no es más que una `Series` de type `bool`, donde esté `True` mostrará el valor y donde esté `False` lo ocultará.

In [37]:
series = pd.Series(["manzana", "mandarina", "kiwi", "pina", "guayaba", "fresa", "fresa", "pina", "manzana", "limon"])

series.head()

0      manzana
1    mandarina
2         kiwi
3         pina
4      guayaba
dtype: object

In [26]:
# primero definimos que queremos ver o no ver, por ejemplo ver solo manzanas
mask = series == "manzana"
mask

0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9    False
dtype: bool

Muestra `True` solo en las posiciones 0 y 8, ahora aplicamos el filtro.

In [27]:
series[mask]

0    manzana
8    manzana
dtype: object

Muchas veces verás escrito el filtro de la siguiente manera:

In [28]:
series[series == "manzana"]

0    manzana
8    manzana
dtype: object

Tal vez tengas interés en no ver las manzanas o incluir más frutas.

In [29]:
# filtrar manzanas fuera
mask = series != "manzana"
series[mask]

1    mandarina
2         kiwi
3         pina
4      guayaba
5        fresa
6        fresa
7         pina
9        limon
dtype: object

In [30]:
# mostrar manzana con kiwi (operador OR) - donde exista manzana o kiwi
mask = (series == "manzana") | (series == "kiwi")
series[mask]

0    manzana
2       kiwi
8    manzana
dtype: object

Importante que cada condicional esté entre paréntesis o no funcionará.

La lista de condicionales son las mismas de python:
```
mayor: >
menor: <
igual: ==
distinto: !=
mayor o igual: >=
menor o igual: <=
```

Para `OR` y `AND` en pandas se debe usar `|` y `&` respectivamente.

Como NumPy también se puede seleccionar por el `index` y hacer _slicing_. No se incluye la última posición.

In [31]:
series[0:3]

0      manzana
1    mandarina
2         kiwi
dtype: object

In [32]:
series[4:8]

4    guayaba
5      fresa
6      fresa
7       pina
dtype: object

In [33]:
series[4]

'guayaba'

In [35]:
series[::-1] # invertir orden

9        limon
8      manzana
7         pina
6        fresa
5        fresa
4      guayaba
3         pina
2         kiwi
1    mandarina
0      manzana
dtype: object

usar `.iloc` funciona igual y es más explícito:

In [38]:
series.iloc[:3] # si dejas vacío antes de los dos puntos, asume 0

0      manzana
1    mandarina
2         kiwi
dtype: object

In [39]:
series.iloc[3:] # si dejas vacio después de los dos puntos asume que es hasta el último

3       pina
4    guayaba
5      fresa
6      fresa
7       pina
8    manzana
9      limon
dtype: object

Si tienes indexado con palabras o fechas también puedes hacer estas operaciones, en este caso la última posición si se incluye.

Hacemos uso de `.loc` para ello.

In [40]:
series = pd.Series(
    ["rojo", "naranja", "verde", "amarillo"],
    name="colores",
    index=["manzana", "mandarina", "kiwi", "pina"],
)
series

manzana          rojo
mandarina     naranja
kiwi            verde
pina         amarillo
Name: colores, dtype: object

In [41]:
series.loc["manzana"]

'rojo'

In [42]:
series.loc["manzana":"kiwi"]

manzana         rojo
mandarina    naranja
kiwi           verde
Name: colores, dtype: object

In [44]:
series.loc["mandarina":]

mandarina     naranja
kiwi            verde
pina         amarillo
Name: colores, dtype: object

In [46]:
series.loc[:"kiwi"]

manzana         rojo
mandarina    naranja
kiwi           verde
Name: colores, dtype: object